In [20]:
import csv
import pandas as pd
import bz2
from collections import namedtuple
from datetime import datetime
from pprint import pprint

## Reading the file using csv module

Here we will read an archive file which is a TAR archive file called bz2 file.

**What Are File Archives ?**:

File compression tools (often called file archivers) like 7-Zip and PeaZip are able to compress one or more files and/or folders to a single file with just one file extension. This makes it much easier to store all of that content in one place or to share multiple files with someone.

The top three most common archive file types are ZIP, RAR, and 7Z

**bz2 file** is a TAR archive, compressed with a Burrows-Wheeler (BZ2) compression algorithm, along with Run-Length Encoding (RLE) for better compression. Most commonly, this file format is used for distributing software packages on Unix based operating systems like Linux.

With the help of python library, bz2 we will read the data file in "rt" mode (to read a file as text).


In [12]:
fileLocation = "C:\\Users\\KVBA\\OneDrive - Ramboll\\Documents\\Literature\\Tutorials\\data ingestion with python\\Exercise Files\Ch02\\02_01\\taxi.csv.bz2"
with bz2.open(fileLocation, "rt") as bz2_file:
    reader = csv.DictReader(bz2_file)

    for csv_record in reader:
        print(csv_record)
        break


{'VendorID': '2', 'tpep_pickup_datetime': '2018-10-31 07:10:55', 'tpep_dropoff_datetime': '2018-11-01 06:43:24', 'passenger_count': '1', 'trip_distance': '2.57', 'RatecodeID': '1', 'store_and_fwd_flag': 'N', 'PULocationID': '211', 'DOLocationID': '48', 'payment_type': '1', 'fare_amount': '14.5', 'extra': '0.5', 'mta_tax': '0.5', 'tip_amount': '4.74', 'tolls_amount': '0.0', 'improvement_surcharge': '0.3', 'total_amount': '20.54'}


This the format of the data stored. We need to define the colums because in csv file the data type is only text. We need to tell the datatypes for each columns:

In [38]:
Column = namedtuple("Column", "src dest convert")

def parse_datetime(text):
    return datetime.strptime(text, "%Y-%m-%d %H:%M:%S")

columns = [
    Column("VendorID", "vendor_id", int),
    Column("passenger_count", "num_passengers", int),
    Column("tip_amount", "tip", float),
    Column("total_amount", "price", float),
    Column("tpep_dropoff_datetime", "drop_time", parse_datetime),
    Column("tpep_pickup_datetime", "pickup_time", parse_datetime),
    Column("trip_distance", "distance", float)
]

def iter_records(file_name):
    with bz2.open(file_name, "rt") as bz2_file:
        reader = csv.DictReader(bz2_file)
        for csv_record in reader:
            record = {}
            for col in columns:
                value = csv_record[col.src]
                record[col.dest] = col.convert(value)
            yield record


def example(nrows=3):
    for i, record in enumerate(iter_records(fileLocation)):
        if i>nrows:
            break
        pprint(record)


Here we have used namedtuple which is nothing but:

```python
## Define the tuple defination
Column = namedtuple("Column", "src dest convert")

## Create an instance:
Column("Vendor ID", "vendor_id", int)
```
Stores the tuple as shown below:
```python
Column(src='Vendor ID', dest='vendor_id', convert=<class 'int'>)
```

In [39]:
example(nrows=4)

{'distance': 2.57,
 'drop_time': datetime.datetime(2018, 11, 1, 6, 43, 24),
 'num_passengers': 1,
 'pickup_time': datetime.datetime(2018, 10, 31, 7, 10, 55),
 'price': 20.54,
 'tip': 4.74,
 'vendor_id': 2}
{'distance': 3.58,
 'drop_time': datetime.datetime(2018, 10, 31, 16, 50, 10),
 'num_passengers': 5,
 'pickup_time': datetime.datetime(2018, 10, 31, 16, 38, 25),
 'price': 13.8,
 'tip': 0.0,
 'vendor_id': 2}
{'distance': 2.39,
 'drop_time': datetime.datetime(2018, 10, 31, 20, 31, 47),
 'num_passengers': 1,
 'pickup_time': datetime.datetime(2018, 10, 31, 20, 23, 41),
 'price': 11.3,
 'tip': 1.0,
 'vendor_id': 2}
{'distance': 0.5,
 'drop_time': datetime.datetime(2018, 10, 31, 22, 48, 28),
 'num_passengers': 1,
 'pickup_time': datetime.datetime(2018, 10, 31, 22, 44, 24),
 'price': 5.8,
 'tip': 0.0,
 'vendor_id': 2}
{'distance': 1.81,
 'drop_time': datetime.datetime(2018, 10, 31, 23, 35, 30),
 'num_passengers': 1,
 'pickup_time': datetime.datetime(2018, 10, 31, 23, 22, 18),
 'price': 13.5

## Reading csv file using pandas

In [43]:
df = pd.read_csv(fileLocation, parse_dates=[1,2])
df.dtypes

VendorID                          int64
tpep_pickup_datetime     datetime64[ns]
tpep_dropoff_datetime    datetime64[ns]
passenger_count                   int64
trip_distance                   float64
RatecodeID                        int64
store_and_fwd_flag               object
PULocationID                      int64
DOLocationID                      int64
payment_type                      int64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
improvement_surcharge           float64
total_amount                    float64
dtype: object

Incase the file size is too big, we can use the chunk size parameter in the pandas:

In [45]:
dfs = pd.read_csv(fileLocation, parse_dates=[1,2], chunksize=1000)
dfs

Now all the df is stored in the textfilereader which contains the dataframe in many chunks:

In [55]:
total_chunks = 0
for sub_df in dfs:
    total_chunks += 1
    print(sub_df.shape)

print("Total number of chunks are", total_chunks)

Total number of chunks are 0
